In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_photo = pd.read_csv('./data/stripe82candidateVar_v1.1.dat',skiprows=5, header=1,index_col=0,sep='\s+')

In [3]:
df_photo.head()

,ra,dec,P,r,ug,gr,ri,iz,gN,gAmpl,rN,rAmpl,iN,iAmpl,zQSO,MiQSO
ID,,,,,,,,,,,,,,,,
7904669,2.501237,-0.150518,2976.182949,18.92,0.11,0.12,0.08,-0.06,59,0.418,63,0.280,61,0.249,1.43,-25.802
7904764,359.999343,0.725718,2.574647,17.26,2.55,1.28,0.73,0.38,29,1.514,29,1.104,29,1.086,-9.90,-9.900
7904816,5.000314,0.034150,3321.033888,19.00,0.01,0.22,-0.05,-0.17,53,0.309,53,0.273,55,0.311,1.13,-24.795
7905184,297.499230,-0.394416,0.444523,14.62,1.90,0.64,0.24,0.11,15,2.046,12,1.046,8,0.045,-9.90,-9.900
7905505,297.498899,-0.247960,1.647168,19.12,0.80,0.38,0.10,-0.05,11,0.116,10,0.112,8,0.157,-9.90,-9.900


In [5]:
colnames = ['ID','Priority','Flags','Teff','eTeff','NTeff','logg','elogg','Nlogg','Z','eZ','NZ']
df_spec = pd.read_csv('./data/t3.txt',sep='\s+',skiprows=26,header=None,index_col=0,names=colnames)

In [7]:
#Combine both datasets 
df = pd.merge(df_spec, df_photo, on='ID',how='left')

In [10]:
df.head()

,Priority,Flags,Teff,eTeff,NTeff,logg,elogg,Nlogg,Z,eZ,...,ri,iz,gN,gAmpl,rN,rAmpl,iN,iAmpl,zQSO,MiQSO
ID,,,,,,,,,,,,,,,,,,,,,
4172970,4.0,nnnnn,5499.7,94.7,7,3.94,0.18,6,0.01,0.12,...,0.13,0.07,29,0.197,27,0.156,27,0.231,-9.9,-9.9
4583821,5.0,Nnnnn,4487.8,120.3,3,4.19,0.26,2,-0.95,0.08,...,0.30,0.16,50,0.124,49,0.106,51,0.094,-9.9,-9.9
4651452,2.0,NnBnX,-9999.0,-9999.0,0,-9999.00,-9999.00,0,-9999.00,-9999.00,...,-0.06,0.26,36,0.505,33,0.460,27,1.150,-9.9,-9.9
4777216,9.0,nnnnX,-9999.0,-9999.0,0,-9999.00,-9999.00,0,-9999.00,-9999.00,...,0.10,0.02,47,0.133,47,0.394,47,0.154,-9.9,-9.9
5302673,9.0,nnnnn,5986.7,161.8,4,3.93,0.79,2,-0.76,0.09,...,0.13,-0.05,36,0.332,33,0.175,39,0.253,-9.9,-9.9


In [32]:
# Cleaning the training set not clearly mentioned in the paper. 
#Usign the entire dataset for now
#df = df[(df['Flags'] != 'nnnnn') & (df[Z] > -3.0)]